## Machine Learning Exercise 3: Bias and Variance

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.pipeline import *
from sklearn.preprocessing import *
from sklearn.impute import *
from sklearn.model_selection import *

**Bias** refers to the error introduced by approximating a complex real-world problem with a simplified model, while **variance** refers to the model's sensitivity to fluctuations in the training data. A linear regression model has high bias and low variance; it makes strong assumptions about the data (linearity) but is stable across different datasets. If these strong assumptions are not correct, there will be places where it systematically overestimates or underestimates. In contrast, a decision tree model has low bias and high variance;it can capture complex patterns but is prone to overfitting, especially if deep and unpruned. This means that it can start to memorize the training data rather than capturing patterns that generalize.

1. Fit a linear regression model to the housing data, using sqft_living to predict price. Check the mean squared error on the training data and the test data.



In [53]:
housing_df = pd.read_csv("../data/kc_house_data.csv")

# specify target and predictor(s)
predictors = ['sqft_living']
target = 'price'

X = housing_df[predictors]
y = housing_df[target]

# train, test, split housing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

# create Linear Regression model
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))
#print("root mean squared error: ", root_mean_squared_error(y_test, y_pred))
#print("mean absolute error: ", mean_absolute_error(y_test, y_pred))
#print("mean absolute percentage error: ", mean_absolute_percentage_error(y_test, y_pred))
#print("R^2: ", r2_score(y_test, y_pred))

mean squared error:  74509993356.49603


2. Repeat this but with a [DecisionTreeRegresor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html). Again check the mean squared error on the training data and the test data. How does what you see differ from the linear regression model?



In [54]:
from sklearn.tree import DecisionTreeRegressor

# create DecisionTreeRegressor model
dtreereg = DecisionTreeRegressor()
dtreereg.fit(X_train, y_train)
y_pred = dtreereg.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))
#print("root mean squared error: ", root_mean_squared_error(y_test, y_pred))
#print("mean absolute error: ", mean_absolute_error(y_test, y_pred))
#print("mean absolute percentage error: ", mean_absolute_percentage_error(y_test, y_pred))
#print("R^2: ", r2_score(y_test, y_pred))

mean squared error:  79044952597.92511


The Decision Tree Regressor does worse than the linear regression model.

One way of avoiding overfitting is by restricting the flexibility of the model. We can do this with a decision tree by restricting the number of splits that it can perform. 

3. Fit a DecisionTreeRegressor where you restrict the max_depth to 5. Again check the mean squared error on the training data and the test data. What do you notice now?



In [55]:
# create DecisionTreeRegressor model with max depth of 5
dtreereg = DecisionTreeRegressor(max_depth = 5)
dtreereg.fit(X_train, y_train)
y_pred = dtreereg.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))
#print("root mean squared error: ", root_mean_squared_error(y_test, y_pred))
#print("mean absolute error: ", mean_absolute_error(y_test, y_pred))
#print("mean absolute percentage error: ", mean_absolute_percentage_error(y_test, y_pred))
#print("R^2: ", r2_score(y_test, y_pred))

mean squared error:  73264250436.88351


The mean squared error is much better!

When working with machine learning models, we often have to balance bias and variance. This is called the [bias-variance tradeoff](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff). One method of this is through [regularization](https://www.ibm.com/think/topics/regularization), where we restrict the complexity of the model, adding some bias but reducing the variance, which can lead to a lower mean squared error on the test set.

Lasso and ridge regression do this by adding a penalty term based on the size of the coefficients. Smaller coefficients means that the model has less flexibility. The neat thing about these types of models is that they determine how to allocate the coefficients automatically as part of the model fitting process, so we can start with a large set of potential predictors and allow the model fitting to determine which ones to focus on.

For the next part of the exercise, we'll see how we can add complexity to our model but control the complexity through regularization.

4. So far, we've only been predicting based off of the square footage of living space. Fit a new linear regression model using all variables besides id, date, price, and zipcode. How well does this model perform on the test set compared to the model with just square footage of living space?



In [56]:
housing_df.head(5)

#categorical = ['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade', ]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [57]:
X = housing_df.drop(columns=['id', 'date', 'price', 'zipcode'])
y = housing_df['price']

# train, test, split housing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

# create Linear Regression model
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))

mean squared error:  44093475276.573814


This model performs significantly better than just using the square footage as a predictor!

5. Try fitting a lasso and ridge model. Becuase lasso and ridge have penalty terms based on the size of the coefficients, and the size of the coefficients depends on the scale of the variable, you'll want to scale the features first so that they are on comparable scales. Create a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) object where the first step is applying a [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and the second step is either a lasso or ridge model. Because these models have a hyperparameter controlling regularization strength, you'll want to use the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) and [RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) models, which will select values for the regularization strength using cross-validation.



In [58]:
pipeline = Pipeline(
    steps = [('scaler', StandardScaler()),
             ('model', LassoCV())]
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))

mean squared error:  44128929521.629326


In [59]:
pipeline = Pipeline(
    steps = [('scaler', StandardScaler()),
             ('model', RidgeCV())]
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))

mean squared error:  44094763932.53586


You likely didn't see much difference between the regular linear regression model and the lasso or ridge model. Let's see what happens if we add more complexity through feature interactions. We can capture more complex relationships between the predictor variables and the target variable by multiplying the predictors together before fitting the model. For example, the interaction between sqft_living and bedrooms will let the model capture if the impact of square footage depends on the number of bedrooms.

6. Add [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to your pipeline after the standard scaler. Try using degree 2 features. How does this change the performance of the regular linear regression model, the lasso model, and the ridge model? 



In [60]:
pipeline = Pipeline(
    steps = [('scaler', StandardScaler()),
             ('pf', PolynomialFeatures(degree=2)),
             ('model', LinearRegression())]
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))

mean squared error:  30926964722.88152


In [61]:
pipeline = Pipeline(
    steps = [('scaler', StandardScaler()),
             ('pf', PolynomialFeatures()),
             ('model', RidgeCV())]
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))

mean squared error:  30933833734.684982


In [62]:
pipeline = Pipeline(
    steps = [('scaler', StandardScaler()),
             ('pf', PolynomialFeatures(degree=2)),
             ('model', LassoCV())]
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))

mean squared error:  30414023904.742867


The lasso penalty tends to cause some coeffients to zero out, so it can be viewed as a method of automatic feature selection.

7. Look at the set of coefficients for the lasso model. What percentage of the coefficients are zero? What are the largest non-zero coefficients?



In [63]:
lasso_model = pipeline.named_steps['model']
coefficients = lasso_model.coef_

print(coefficients)
print("Percent nonzero coefficients: ", np.count_nonzero(coefficients)/len(coefficients))
print("Percent zero coefficients: ", 1 - (np.count_nonzero(coefficients)/len(coefficients)))
print(max(coefficients))

[ 0.00000000e+00 -0.00000000e+00  1.29631038e+04  9.30898037e+04
 -0.00000000e+00 -7.40640157e+03  0.00000000e+00  0.00000000e+00
  2.55901284e+04  9.49584662e+04  3.48881548e+03  0.00000000e+00
 -1.89758098e+04  0.00000000e+00  6.91512974e+04 -2.23709363e+04
  4.64911392e+04 -0.00000000e+00  1.34675819e+02 -3.42301923e+03
 -5.54586218e+03  0.00000000e+00  0.00000000e+00 -8.21509899e+02
 -0.00000000e+00  2.68731640e+02  4.54886920e+02 -0.00000000e+00
 -7.66847183e+02 -4.16932354e+02 -2.08144768e+03 -0.00000000e+00
  5.53855599e+03  2.71218569e+02  1.37786566e+03 -0.00000000e+00
  1.94692202e+04 -3.77436715e+03 -5.05144532e+03 -0.00000000e+00
  2.70220671e+03 -1.27452732e+03  9.42420011e+02  7.47394194e+03
  0.00000000e+00  3.62574286e+03 -4.04970904e+03  0.00000000e+00
 -1.84220237e+03 -6.94222056e+02  0.00000000e+00  4.36724451e+03
 -5.07516407e+03 -0.00000000e+00  6.50325965e+03  0.00000000e+00
  0.00000000e+00  3.34707747e+04  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.12849

In [64]:
coefficients = lasso_model.coef_
coefficients = pd.DataFrame({
    'variable': pipeline[:-1].get_feature_names_out(),
    'coefficient': pipeline['lasso'].coef_
})

KeyError: 'lasso'

8. A new hyperparameter that we have is the degree of the polynomial we're using. So that we're not overfitting to the test set, we need to use cross-validation to select this value. Set up a [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to try out polynomial degrees from 1 to 3 and to try LinearRegression, LassoCV, and RidgeCV models. Use 'neg_mean_squared_error' as the error_score. Which combination does it find does the best? 



In [ ]:
pipeline = Pipeline(
    steps = [('scaler', StandardScaler()),
             ('pf', PolynomialFeatures()),
             ('model', LinearRegression())]
)

cv_pipe = GridSearchCV(pipeline,
                       param_grid={'pf__degree': [1,2,3]},
                       scoring='neg_mean_squared_error',
                       n_jobs = -1,
                       cv = 5)

cv_pipe.fit(X_train, y_train)
print(cv_pipe.best_params_)
y_pred = cv_pipe.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))

{'pf__degree': 2}
mean squared error:  30926964722.88152


In [ ]:
pipeline = Pipeline(
    steps = [('scaler', StandardScaler()),
             ('pf', PolynomialFeatures()),
             ('model', RidgeCV())]
)

cv_pipe = GridSearchCV(pipeline,
                       param_grid={'pf__degree': [1,2,3]},
                       scoring='neg_mean_squared_error',
                       n_jobs = -1,
                       cv = 5)

cv_pipe.fit(X_train, y_train)
print(cv_pipe.best_params_)
y_pred = cv_pipe.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))

{'pf__degree': 2}
mean squared error:  30933833734.684982


In [ ]:
pipeline = Pipeline(
    steps = [('scaler', StandardScaler()),
             ('pf', PolynomialFeatures()),
             ('model', LassoCV())]
)

cv_pipe = GridSearchCV(pipeline,
                       param_grid={'pf__degree': [1,2,3]},
                       scoring='neg_mean_squared_error',
                       n_jobs = -1,
                       cv = 5)

cv_pipe.fit(X_train, y_train)
print(cv_pipe.best_params_)
y_pred = cv_pipe.predict(X_test)

print("mean squared error: ", mean_squared_error(y_test, y_pred))

{'pf__degree': 2}
mean squared error:  30414023904.742867


In [68]:
from itertools import product

degrees = [1, 2, 3]
models = [LinearRegression(), LassoCV(max_iter = 10000), RidgeCV()]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
best_score = float("-inf")
best_params = None
for degree, model in product(degrees, models):
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("poly", PolynomialFeatures(degree=degree)),
        ("model", model)
    ])
    
    pipeline.fit(X_train, y_train)
    score = pipeline.score(X_test, y_test)
    if score > best_score:
        best_score = score
        best_params = {"degree": degree, "model": model}
    print(f"Test score for degree={degree}, model={model}: {score:.4f}")
print("\nBest parameters:", best_params)
print("Best score:", best_score)

Test score for degree=1, model=LinearRegression(): 0.6946
Test score for degree=1, model=LassoCV(max_iter=10000): 0.6943
Test score for degree=1, model=RidgeCV(): 0.6946
Test score for degree=2, model=LinearRegression(): 0.7858
Test score for degree=2, model=LassoCV(max_iter=10000): 0.7893
Test score for degree=2, model=RidgeCV(): 0.7857
Test score for degree=3, model=LinearRegression(): -2535533486043997.5000
Test score for degree=3, model=LassoCV(max_iter=10000): 0.7711
Test score for degree=3, model=RidgeCV(): -0.2823

Best parameters: {'degree': 2, 'model': LassoCV(max_iter=10000)}
Best score: 0.7893277370558397


In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
pipeline = Pipeline([
    ("scaler", StandardScaler()),    # Standardize features
    ("poly", PolynomialFeatures()),  # Placeholder for polynomial transformation
    ("model", LinearRegression())    # Placeholder for model
])
param_grid = {
    "poly__degree": [1, 2, 3],  # Different polynomial degrees
    "model": [LinearRegression(), LassoCV(max_iter = 10000), RidgeCV()],  # Different models
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1, verbose = 2)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best model:", grid_search.best_estimator_)
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters: {'model': LinearRegression(), 'poly__degree': 2}
Best model: Pipeline(steps=[('scaler', StandardScaler()), ('poly', PolynomialFeatures()),
                ('model', LinearRegression())])


** If you've reached this point, let your instructors know so that they can check in with you. **

Stretch Goals:

1. With home prices, there are some extremely large values for price, which can overly-influence the mean squared error calculation. See what happens if you optimize for mean absolute error instead. Alternatively, try using a [TransformedTargetRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html) to predict the log of price instead of the raw price.

**Bonus Exercise** We've seen how a decision tree model has move flexibility, which mean higher variance compared to a linear regression model. One way of understanding variance is that variance describes how sensitive the model is to the training data. A model with high variance can produce vastly different predictions when trained on different subsets.

In this bonus exercise, you'll get to see this in action.

Generate 1000 different linear regression fits, where you only use sqft_living as the predictor variable. For each fit, choose a random sample from the full dataset of size 1000. 
Using the np.linspace function, generate a grid of 100 equally-spaced points between 500 and 3000 and generate predictions on those points. Plot the mean prediction, the 5th percentile, and the 95th percentile for the predictions from these thousand models. Repeat this for a decision tree model. Then do it for a decision tree model with a max_depth of 5.

How do these predictions differ? Which have the most variability?